In [1]:
import onnx_graphsurgeon as gs
import onnx
import numpy as np

In [2]:
graph = gs.import_onnx(onnx.load("trans_modified.onnx"))

In [9]:
graph

[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _ipython_display_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_mimebundle_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_html_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_markdown_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_svg_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_png_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_pdf_ registered for opset: 11
[E] No function: _ipython_ca

Graph torch-jit-export (Opset: 11)
Inputs: [Variable (input.1): (shape=[1, 512, 180, 180], dtype=float32)]
Nodes:
Conv_0 (Conv)
	Inputs: [
		Variable (input.1): (shape=[1, 512, 180, 180], dtype=float32)
		Constant (shared_conv.weight): (shape=[128, 512, 3, 3], dtype=<class 'numpy.float32'>)
		Constant (shared_conv.bias): (shape=[128], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (98): (shape=[1, 128, 180, 180], dtype=float32)
	]
Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [3, 3]), ('pads', [1, 1, 1, 1]), ('strides', [1, 1])])
Div_127 (Div)
	Inputs: [
		Variable (283): (shape=[1, 200], dtype=int64)
		Constant (285): (shape=[], dtype=<class 'numpy.int64'>)
	]
	Outputs: [
		Variable (286): (shape=[1, 200], dtype=int64)
	]
Div_131 (Div)
	Inputs: [
		Variable (283): (shape=[1, 200], dtype=int64)
		Constant (285): (shape=[], dtype=<class 'numpy.int64'>)
	]
	Outputs: [
		Variable (290): (shape=[1, 200], dtype=int64)
	]
Reshape_1 (Reshape)
	Inputs

In [4]:
def Add_plugin(graph):
    plugin = gs.Node(op = "MyTopKPlugin_TRT", name = "MyTopKPlugin_001")
    graph.nodes.append(plugin)
    wp_input_tensor = [node.outputs[0] for node in graph.nodes if node.name == "Reshape_115"][0]
    wp_output_tensor = [node.inputs[0] for node in graph.nodes if node.name == "Div_131"][0]
    wp_input_tensor.outputs[0] = plugin
    wp_output_tensor.inputs[0] = plugin

In [5]:
Add_plugin(graph)

In [3]:
def Modify_plugin(graph):
    values = gs.Variable(name="topk_values", dtype=np.float32, shape=(1, 200))
    indices = gs.Variable(name="topk_indices", dtype=np.int32, shape=(1, 200))
    for node in graph.nodes:
        if node.op == "MyTopKPlugin_TRT":
            node.outputs = [values, indices]
        if node.name == "Div_127":
            node.inputs[0] = indices
        if node.name == "Div_131":
            node.inputs[0] = indices
        if node.name == "Sub_134":
            node.inputs[0] = indices

In [4]:
Modify_plugin(graph)

In [5]:
def Add_outputs(graph):
    for node in graph.nodes:
        if node.op == "MyTopKPlugin_TRT":
            graph.outputs.append(node.outputs[0])
            graph.outputs.append(node.outputs[1])

In [6]:
Add_outputs(graph)

In [3]:
def Add_reshape(graph):
    error_tensor = gs.Variable(name="error_tensor", dtype=np.int64, shape=(1, 10, 200))
    error_tensor0 = gs.Variable(name="error_tensor0", dtype=np.int64, shape=(1, 10, 200))
    for node in graph.nodes:
        if node.op == "OneHot1DPlugin_TRT":
            node.outputs = [error_tensor]
        if node.name == "Cast_147":
            node.inputs = [error_tensor0]
    reshape = gs.Node(op="Reshape", name="reshape_plugin")
    graph.nodes.append(reshape)
    shape = gs.Constant("reshape", np.array(error_tensor.shape))
    reshape.inputs = [error_tensor, shape]
    reshape.outputs = [error_tensor0]

In [4]:
Add_reshape(graph)

In [7]:
graph.cleanup().toposort()

[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _ipython_display_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_mimebundle_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_html_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_markdown_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_svg_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_png_ registered for opset: 11
[E] No function: _ipython_canary_method_should_not_exist_ registered for opset: 11
[E] No function: _repr_pdf_ registered for opset: 11
[E] No function: _ipython_ca

Graph torch-jit-export (Opset: 11)
Inputs: [Variable (input.1): (shape=[1, 512, 180, 180], dtype=float32)]
Nodes:
Conv_0 (Conv)
	Inputs: [
		Variable (input.1): (shape=[1, 512, 180, 180], dtype=float32)
		Constant (shared_conv.weight): (shape=[128, 512, 3, 3], dtype=<class 'numpy.float32'>)
		Constant (shared_conv.bias): (shape=[128], dtype=<class 'numpy.float32'>)
	]
	Outputs: [
		Variable (98): (shape=[1, 128, 180, 180], dtype=float32)
	]
Attributes: OrderedDict([('dilations', [1, 1]), ('group', 1), ('kernel_shape', [3, 3]), ('pads', [1, 1, 1, 1]), ('strides', [1, 1])])
Reshape_1 (Reshape)
	Inputs: [
		Variable (98): (shape=[1, 128, 180, 180], dtype=float32)
		Constant (646): (shape=[3], dtype=<class 'numpy.int64'>)
	]
	Outputs: [
		Variable (105): (shape=[1, 128, 32400], dtype=float32)
	]
Conv_7 (Conv)
	Inputs: [
		Variable (98): (shape=[1, 128, 180, 180], dtype=float32)
		Constant (617): (shape=[128, 128, 3, 3], dtype=<class 'numpy.float32'>)
		Constant (618): (shape=[128], dtype=<

In [8]:
onnx.save_model(gs.export_onnx(graph), "trans_modified1.onnx")

In [37]:
graph.nodes[458]

 (WindowProcessPlugin)
	Inputs: [
		Variable (373): (shape=None, dtype=None)
	]
	Outputs: [
		Variable (413): (shape=None, dtype=None)
	]

In [7]:
!python3 -m pip install colored

  Preparing metadata (setup.py) ... done
  Created wheel for colored: filename=colored-1.4.4-py3-none-any.whl size=14269 sha256=5c09ae4d026c09d0d15030239399773929b22c4f3daacfaf89ab685a5d3e6a7a
  Stored in directory: /root/.cache/pip/wheels/95/b9/be/9472bb50db786f327678fa8e575fce2c268593118a9257a722
Successfully built colored
